## Straturi Noi

In continuare o sa utilizam o parte din straturile prezentate in curs.

Staturi noi:

Layer Convolutional:
* torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)

Layere Pooling:
* torch.nn.MaxPool2d(kernel_size, stride=None, padding=0)
* torch.nn.AveragePool2d(kernel_size, stride=None, padding=0)

Layere Adaptive Pool, intalnit adesea si ca Global Pool:
* torch.nn.AdaptiveAvgPool2d(output_size)
* torch.nn.AdaptiveMaxPool2d(output_size)

Layer de liniarizare:

* torch.nn.Flatten()



In [ ]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(2,2))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(13,13), stride=(2,2))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d(kernel_size=(3,3)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer(dummy_input_tensor).shape)

# Utilizate pentru a reduce dimensiunea la una prestabilita, util cand marimea input ului este variabil
layer = nn.AdaptiveAvgPool2d(output_size=(5,5))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

layer = nn.Flatten()
print("Flaten result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 49, 49])
Conv2 result shape torch.Size([1, 10, 44, 44])
Pool result shape torch.Size([1, 3, 33, 33])
Global Pool result shape torch.Size([1, 3, 5, 5])
Flaten result shape torch.Size([1, 30000])


###Cerinte

**(1p)** Utilizati o serie de Conv2D/Pool2D pentru a ajunge la urmatoarele marimi plecand de la input 3x100x100:
*   [1, 10, 24, 24]
*   [1, 10, 9, 9]
*  [1, 3, 2, 2]



In [ ]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

## Instantierea seturilor de date

In [ ]:
import torchvision

cifar_train = torchvision.datasets.CIFAR10("./data", download=True)
cifar_test = torchvision.datasets.CIFAR10("./data", train=False)

Files already downloaded and verified


## Crearea Dataloader-ului

### Cerinte
 * **(2p)** Implementati functia de preprocesare a datelor, __preproc_fn(examples)__.


Atentie! Spre deosebire de intrarea pentru retelele fully-connected, pentru retelele convolutionale intrearea nu trebuie liniarizata, ci doar normalizata.

#### Hint

  * Amintiti-va folosirea functiei __normalize__ din torchvision.transforms.functional din laboratorul trecut.
  * Modificati functia *preproc_fn* din laboratorul trecut, pentru a normaliza datele in intervalul [-1, 1]

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataloader
from torchvision.transforms.functional import to_tensor, normalize

def preproc_fn(examples):
  ### Completati codul pentru cerinta aici
  return examples


train_loader = Dataloader(cifar_train, batch_size=500, shuffle=True, num_workers=2, collate_fn=preproc_fn)
test_loader = Dataloader(cifar_test, batch_size=1, suffle=False, collate_fn=preproc_fn)

## Crearea unei retele neurale convolutionale

### Cerinte
 * **(1p)** Creati o clasa care mosteneste clasa nn.Module. Ea va reprezenta o retea neurala convolutionala pentru clasificare ale celor 10 clase din datasetul CIFAR10.
    * Reteaua trebuie sa aiba 2 straturi convolutionale care sa reduca dimensiunea spatiala a imaginii de 2 ori
    * Liniarizati iesirea din cel de-al doilea strat convolutional
    * Adaugat stratul final de tipul 'fully-connected'
    * Folositi o functie de activare la alegere

#### Hint

Pentru a liniariza iesirea din cel de-al doilea feature map puteti adopta mai multe strategii:
  * Liniarizare prin schimbarea shape-ului la [batch_size, -1]
  * Global Max Pooling si apoi liniarizare la [batch_size, -1]
  * Average Max Pooling si apoi liniarizare la [batch_size, -1]

In [ ]:
import torch.nn as nn

class Net(nn.Module):
  def __init__(self):
    pass

  def forward(self,x):
    return x

## Definirea obiectelor folosite in timpul antrenarii

### Cerinte **(1p)**
  * Numarul de epoci
  * Retea
  * Optimizator
  * Alegeti functia de cost

In [ ]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = None

# Definiti reteaua
network = None

# Definiti optimizatorul
optimizer = None
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = 

## Definirea functiei de antrenare

In [ ]:
def train_fn(epochs: int, train_loader: data.DataLoader, test_loader: data.DataLoader, 
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  # Iteram prin numarul de epoci
  for e in range(epochs):
    # Iteram prin fiecare exemplu din dataset
    for images, labels in train_loader:

      # Aplicam reteaua neurala pe imaginile de intrare
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe adnotarile imaginilor 
      loss = loss_fn(out, labels)
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a aplica gradientii pe parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()
    
    print("Loss-ul la finalul epocii {} are valoarea {}".format(e, loss.item()))

    # Caluculul acuratetii
    count = len(test_loader)
    correct = 0

    for test_image, test_label in test_loader:
      out_class = torch.argmax(net(test_image))
      if out_class == test_label:
        correct += 1

    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e, (correct / count) * batch_size))


## Antrenarea

### Cerinte
  * Antrenati reteaua definita mai sus (clasa Net)

In [ ]:
train_fn(epochs, train_loader, test_loader, network, loss_fn, optimizer)

## Reteaua LeNet

### Cerinte
  * **(3p)** Implementati reteaua LeNet dupa figura de mai jos si antrenati-o


![alt text](https://drive.google.com/uc?id=1OVancUyIViMRMZdULFSVCvXJHQP0NGUV)

Figura arhitectura LeNet

![alt text](https://debuggercafe.com/wp-content/uploads/2019/07/Layers-in-LeNet.png)

Tabel arhitectura LeNet


In [ ]:
import torch.nn as nn

class LeNet(nn.Module):
  def __init__(self):
    pass

  def forward(self,x):
    return x

## Redefinirea obiectelor folosite in timpul antrenarii pentru reteaua LeNet

### Cerinta
 * Redefiniti obiectele pentru a antrena reteaua LeNet

In [ ]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = None

# Definiti reteaua
lenet = None

# Definiti optimizatorul
lenet_optimizer = None
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = 

## Antrenarea retelei LeNet

In [ ]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

###Augmentare retea

Reteaua de mai devreme duce lipsa de regularizare. O forma foarte puternica de regularizare este normalizarea, iar pentru acest lucru exista straturi speciale.

Astfel de straturi:

* torch.nn.BatchNorm2d(num_features)
* torch.nn.InstanceNorm2d(num_features)

Un alt element important il reprezinta functiile de activare, care pot influenta convergenta si puterea retelei. Cateva exemple de functii de activate:


* Relu
* Sigmoid
* Tanh
* LeakyRelu
* GELU

## Cerinta

**(2p)** Experimentati cu aceste elemente in cadrul retelei LeNet definita mai devreme, pentru a obtine o acuratete mai buna. Observati viteza de convergenta si performanta retelei pentru 3 configuratii diferite.


###Bonus
**(1p)** Antrenati reteaua folosind GPU (Graphics processing unit)








  
